## Install libraries and packages

### Install packages

In [5]:
# Instalar las bibliotecas necesarias
%pip install pandas fuzzywuzzy python-Levenshtein pytesseract opencv-python pdf2image tqdm numpy requests Pillow
# Instalar Tesseract OCR


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Import libraries

In [ ]:
import pandas as pd
import time
import numpy as np
from tqdm.auto import tqdm


In [2]:
import re
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from fuzzywuzzy import process
import unicodedata
import copy

In [ ]:
"""import pytesseract as pyt
pyt.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'"""
# Configure Tesseract OCR path 

In [4]:

import cv2
from pdf2image import convert_from_path
from PIL import Image, ImageOps
from concurrent.futures import ThreadPoolExecutor

### Mount Google Drive

In [ ]:
""" Mount Drive
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') """ # Uncomment if using Google Colab


Mounted at /content/drive


## Make subset 'veto_dataframe' and merge with asambleistas

In [ ]:
# Load the cleaned DataFrame
csv_path = '/content/drive/MyDrive/Colab Notebooks/cleaned_merge_df.csv' # Update this path to your actual CSV file location
df_clean = pd.read_csv(csv_path)
len(df_clean)
df_clean.columns.tolist()
#From now on we work with the subset of bills approved after 2nd debate
df_subset = df_clean[(df_clean['texto_aprobado_pleno_pdf_presente'] == 1) & ((df_clean['objecion_parcial_pdf_presente'] == 1) | (df_clean['objecion_total_pdf_presente'] == 1))]
len(df_subset)

In [ ]:
#Check number of downloaded files
list(df_subset)

cols = [
    'texto_aprobado_pleno_pdf_presente',
    'objecion_parcial_pdf_presente',
    'objecion_total_pdf_presente',
    'texto_definitivo_pleno_pdf_presente',
    'registro_oficial_pdf_presente'
]

counts = df_subset[cols].eq(1).sum()
counts
df_subset.head()
#619+512+107+493+487

#2052 valid documents out of 2218

,fecha_presentacion,proyecto_ley,id,proponente,cargo,coalition_x,party_2_x,comision,estado,texto_proyecto_ley_pdf_presente,...,objecion_total,texto_definitivo_pleno_pdf_presente,texto_definitivo_pleno,registro_oficial_pdf_presente,registro_oficial,termino_legislativo,matched_name,matching_score,coalition_y,party_2_y
0,2024-03-20,Proyecto de Ley Reformatoria a varios cuerpos ...,AN-2024-2794,veloz ramirez rebeca viviana,ASAMBLEÍSTA/S,UNES,RC,Comisión de Justicia y Estructura del Estado,Archivado / Registro de Objeción Total,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,0,NaN,5.0,veloz ramirez rebeca viviana,100.0,UNES,RC
1,2024-03-08,Proyecto de Ley Reformatoria a la Ley Orgánica...,AN-2024-2786,castillo campoverde alexandra andrea,ASAMBLEÍSTA/S,NaN,NaN,Comisión de Justicia y Estructura del Estado,En trámite / Revisión en Comisión para Primer ...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,0,NaN,5.0,NaN,NaN,NaN,NaN
2,2024-03-05,Proyecto de Ley Orgánica de Inteligencia,AN-2024-2776,alarcon bueno ines margarita,ASAMBLEÍSTA/S,NaN,NaN,"Comisión de Soberanía, Integración y Seguridad...",En trámite / Revisión en Comisión para Primer ...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,0,NaN,5.0,NaN,NaN,NaN,NaN
3,2024-02-15,Proyecto de Ley Orgánica Reformatoria al Códig...,AN-2024-2757,castro piedra adrian ernesto,ASAMBLEÍSTA/S,NaN,NaN,Comisión de Gobiernos Autónomos Descentralizac...,En trámite / Revisión en Comisión para Primer ...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,0,NaN,5.0,NaN,NaN,NaN,NaN
4,2024-01-11,Proyecto de Ley Orgánica Reformatoria al Códig...,AN-2024-2713,samaniego maigua franklin omar,ASAMBLEÍSTA/S,NaN,NaN,Comisión de Justicia y Estructura del Estado,En trámite / Revisión en Comisión para Primer ...,1,...,NaN,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,5.0,samaniego maigua franklin omar,98.0,NaN,NaN


In [ ]:
#Load asambleistas dataset
import unicodedata
df_asambleistas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/asambleistas_09_24.csv') # Update this path to your actual CSV file location
df_asambleistas.head()

#Transform names to lowercase and remove accents

#Function
def remove_accents(text: str) -> str:
    """Strip accents from input string."""
    if not isinstance(text, str):
        return text
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )

#Apply function
df_asambleistas['name'] = (
    df_asambleistas['name']
    .str.lower()
    .apply(remove_accents)
)

df_asambleistas.head()
# df_asambleistas.to_csv('/content/drive/MyDrive/Colab Notebooks/asambleistas_09_24.csv', index=False) # Uncomment to save the cleaned dataset


In [ ]:
#Remove accents and transform letters to lowercase
#Function
def remove_accents(text: str) -> str:
    """Strip accents from input string."""
    if not isinstance(text, str):
        return text
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )

#Apply function
df_subset.loc[:, 'proponente'] = (
    df_subset['proponente']
    .str.lower()
    .apply(remove_accents)
)

df_subset.head(10)
# df_subset.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_pl.csv', index=False) # Uncomment to save the subset DataFrame

In [ ]:
#Create termino_legislativo based on fecha_presentacion
conds = [
    df_subset['fecha_presentacion'].between('2009-05-21','2013-05-13'),
    df_subset['fecha_presentacion'].between('2013-05-14','2017-05-11'),
    df_subset['fecha_presentacion'].between('2017-05-12','2021-05-13'),
    df_subset['fecha_presentacion'].between('2021-05-14','2023-05-17'),
    df_subset['fecha_presentacion'] > '2023-05-18'
]
terms = [1, 2, 3, 4, 5]


print(df_subset.columns)

df_subset.loc[:,'termino_legislativo'] = np.select(conds, terms, default=np.nan)
# df_subset.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_pl.csv', index=False) # Uncomment to save the updated subset DataFrame
df_subset.sample(5)

Index(['fecha_presentacion', 'proyecto_ley', 'id', 'proponente', 'cargo',
       'comision', 'estado', 'texto_proyecto_ley_pdf_presente',
       'texto_proyecto_ley', 'texto_1debate_pdf_presente', 'texto_1debate',
       'texto_2debate_pdf_presente', 'texto_2debate',
       'texto_aprobado_pleno_pdf_presente', 'texto_aprobado_pleno',
       'objecion_parcial_pdf_presente', 'objecion_parcial',
       'objecion_total_pdf_presente', 'objecion_total',
       'texto_definitivo_pleno_pdf_presente', 'texto_definitivo_pleno',
       'registro_oficial_pdf_presente', 'registro_oficial',
       'termino_legislativo'],
      dtype='object')


,fecha_presentacion,proyecto_ley,id,proponente,cargo,comision,estado,texto_proyecto_ley_pdf_presente,texto_proyecto_ley,texto_1debate_pdf_presente,...,texto_aprobado_pleno,objecion_parcial_pdf_presente,objecion_parcial,objecion_total_pdf_presente,objecion_total,texto_definitivo_pleno_pdf_presente,texto_definitivo_pleno,registro_oficial_pdf_presente,registro_oficial,termino_legislativo
2150,2014-03-24,Ley Orgánica de Fomento al Emprendimiento,AN-2014-106,mae montano valencia,ASAMBLEÍSTA/S,"Comisión de Desarrollo Económico, Productivo y...",Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,2.0
2298,2012-04-24,Proyecto de Ley Reformatorio a los Arts. 50 y ...,AN-2014-279,paco fierro oviedo,ASAMBLEÍSTA/S,Comisión de Gobiernos Autónomos Descentralizac...,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,1.0
1977,2016-06-21,Proyecto de Ley Orgánica Reformatoria al Códig...,AN-2016-854,"rosana alvarado carrion, mariangel munoz vicun...",ASAMBLEÍSTA/S,Comisión de Justicia y Estructura del Estado,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,2.0
1736,2017-10-31,Proyecto de Ley Reformatoria al Código Orgánic...,AN-2017-1118,zambrano ortiz maria roberta,ASAMBLEÍSTA/S,Comisión de Justicia y Estructura del Estado,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,3.0
2633,2010-01-13,Proyecto de Ley Derogatoria No.1 Para la Depur...,AN-2014-442,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,Comisión de Justicia y Estructura del Estado,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,0,NaN,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,https://ppless.asambleanacional.gob.ec/alfresc...,1.0


### Add coalition and party from 'asambleistas'

In [ ]:
# Add coalition and party to the dataframe, matched_name and matching_score added for authenticity
# Read CSVs
df_subset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/subset_pl.csv') # Update this path to your actual CSV file location
df_asambleistas = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/asambleistas_09_24.csv') # Update this path to your actual CSV file location

# Select relevant columns from df_asambleistas and ensure unique names for lookup
df_asambleistas_subset = df_asambleistas[['name', 'coalition', 'party_2']].drop_duplicates(subset='name', keep='first')


df_subset['proponente'] = df_subset['proponente'].apply(lambda x: remove_accents(str(x).lower()))
df_asambleistas_subset['name'] = df_asambleistas_subset['name'].apply(lambda x: remove_accents(str(x).lower()))

# Get the list of asambleista names for matching
asambleista_names = df_asambleistas_subset['name'].tolist()

# Create columns for the matched asambleista name and matching score
df_subset['matched_name'] = np.nan
df_subset['matching_score'] = np.nan # Optional: store the matching score

# Iterate through each row in df_subset to find the best match
for index, row in tqdm(df_subset.iterrows(), total=df_subset.shape[0], desc="Matching Proponente Names"):
    proponente_name = row['proponente']

    # Handle cases where proponente is not a valid string
    if not isinstance(proponente_name, str) or proponente_name.strip() == '' or proponente_name == 'nan':
        continue

    # Find the best match in the list of asambleista names
    best_match = process.extractOne(proponente_name, asambleista_names)

    if best_match:
        matched_name, score = best_match

        # Set a threshold for considering a match valid (adjust as needed)
        if score >= 80: # less then 90 can also take into acount only the last name  or surname
            df_subset.loc[index, 'matched_name'] = matched_name
            df_subset.loc[index, 'matching_score'] = score


# Rename 'name' in asambleista subset to 'matched_name' to merge
df_asambleistas_for_merge = df_asambleistas_subset.rename(columns={'name': 'matched_name'})

# Merge df_subset with a subset of df_asambleistas based on the matched_name
df_subset = pd.merge(
    df_subset,
    df_asambleistas_for_merge[['matched_name', 'coalition', 'party_2']],
    on='matched_name',
    how='left'
)

# Save dataframe
# df_subset.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_pl.csv', index=False) # Uncomment to save the updated subset DataFrame with coalition and party information



Matching Proponente Names:   0%|          | 0/619 [00:00<?, ?it/s]

<ipython-input-41-015929ba6104>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'veloz ramirez rebeca viviana' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_subset.loc[index, 'matched_name'] = matched_name


,fecha_presentacion,proyecto_ley,id,proponente,cargo,comision,estado,texto_proyecto_ley_pdf_presente,texto_proyecto_ley,texto_1debate_pdf_presente,...,registro_oficial,termino_legislativo,matched_name,matching_score,coalition_x,party_2_x,coalition_y,party_2_y,coalition,party_2
194,2019-06-11,Proyecto de Ley Reformatoria a la Ley para el ...,AN-2019-1610,arteaga munoz karina cecilia,ASAMBLEÍSTA/S,Comisión del Derecho a la Salud y Deporte,Archivado / Registro de Objeción Total,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,NaN,3.0,arteaga munoz karina cecilia,98.0,Sin bancada,Ind.,Sin bancada,Ind.,Sin bancada,Ind.
126,2020-12-21,Proyecto de Ley Orgánica Reformatoria al Códig...,AN-2020-1975,duran aguilar liliana elizabeth,ASAMBLEÍSTA/S,Comisión de Biodiversidad y Recursos Naturales,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,3.0,duran aguilar liliana elizabeth,100.0,Revolución Ciudadana,RC,Revolución Ciudadana,RC,Revolución Ciudadana,RC
371,2017-10-19,Proyecto de Ley Orgánica Reformatoria a la Ley...,AN-2017-1106,zoila teresa benavides zambrano,ASAMBLEÍSTA/S,"Comisión de Educación, Cultura, Ciencia, Tecno...",Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,3.0,benavides zambrano zoila teresa,95.0,PAIS,PAIS,PAIS,PAIS,PAIS,PAIS
499,2013-02-21,Proyecto de Ley Reformatoria a la Ley de Perso...,AN-2014-123,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,Comisión de Relaciones Internacionales y Movil...,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,1.0,correa delgado pierina sara mercedes,86.0,UNES,RC,UNES,RC,UNES,RC
79,2021-10-06,Proyecto de Ley Orgánica de Atención Prioritar...,AN-2021-2132,jimenez torres bella daniela,ASAMBLEÍSTA/S,Comisión del Derecho a la Salud y Deporte,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,https://ppless.asambleanacional.gob.ec/alfresc...,4.0,jimenez torres bella daniela,100.0,Sin bancada,Ind.,Sin bancada,Ind.,Sin bancada,Ind.


In [ ]:
# Print
df_subset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/subset_con_partido.csv')
df_subset[df_subset['cargo'] == 'ASAMBLEÍSTA/S'][[
    'fecha_presentacion', 'proponente', 'matched_name', 'matching_score', 'coalition', 'party_2'
]]

nan_count = df_subset['matched_name'].isna()
print(nan_count)
df_subset_nan = df_subset[df_subset['matched_name'].isna()]
df_subset_nan.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_nan.csv', index=False)
df_subset_nan


0      False
1      False
2       True
3      False
4      False
       ...  
614    False
615    False
616    False
617    False
618     True
Name: matched_name, Length: 619, dtype: bool


NameError: name 'sdvsdvs' is not defined

## Add pdf text to dataset with OCR


### Functions


#### OCR Function

In [ ]:
# Original
def extract_text_pdf(pdf_path_to_extract):
  if not os.path.exists(pdf_path_to_extract):
          # Warining if path don't exists
          return None

  try:
      # Convert PDF to a list of images (one image per page)
      images = convert_from_path(pdf_path_to_extract)

      # Extract text from each image (page)
      extracted_text = ""
      for i, img in enumerate(images):
          # Use pytesseract.image_to_string on the Image object
          page_text = pyt.image_to_string(img, lang='spa') # Spanish
          extracted_text += f"--- Page {i+1} ---\n{page_text}\n\n"
      return extracted_text

  except FileNotFoundError:
      print(f"Error: The PDF file was not found at {pdf_path_to_extract}")
  except Exception as e:
      print(f"An error occurred during PDF processing: {e}")


def insert_ocr_texts(df, pdf_dir):
    """
    Insert ocr columns with none values. Iterates trough dataframe extracting text from
    PDFs using the function 'extract_text_pdf'
    - df is the dataframe with the pdfs links and where the text shold be inserted
    - pdf_dir is the directory where the pdfs are already downloaded
    """
    pdf_columns = [
        'texto_2debate',
        'texto_aprobado_pleno', 'objecion_parcial', 'objecion_total',
        'texto_definitivo_pleno', 'registro_oficial'
    ]

    # New text columns
    ocr_columns = [f"{col}_ocr" for col in pdf_columns]

    for ocr_col in ocr_columns:
        if ocr_col not in df.columns:
            df[ocr_col] = None

    # Iterate through each DataFrame row with a progress bar.
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Extracting OCR text"):
        for column in pdf_columns:
            flag_col = f"{column}_pdf_presente"
            ocr_col = f"{column}_ocr"

            # Verify if pdf is present
            if flag_col in row and row[flag_col] == 1 and (pd.isna(row[ocr_col]) or row[ocr_col] == ''):

                file_name = f"{row.get('id', 'no_id')}_{column}.pdf"
                file_path = os.path.join(pdf_dir, file_name)

                # Text extraction
                text = extract_text_pdf(file_path)

                # Insert text in columns
                if text is not None:
                   df.at[index, ocr_col] = text
                   tqdm.write(f"OCR successful for {file_name}")
                else:
                   tqdm.write(f"OCR failed or file not found for {file_name}")
                   df.at[index, ocr_col] = None

    return df

#### OCR Function saving the dataframe simultaneously

In [ ]:
# Saving dataframe at the same time
def extract_text_pdf(pdf_path_to_extract):
  if not os.path.exists(pdf_path_to_extract):
          # Warining if path don't exists
          return None

  try:
      # Convert PDF to a list of images (one image per page)
      images = convert_from_path(pdf_path_to_extract)

      # Extract text from each image (page)
      extracted_text = ""
      for i, img in enumerate(images):
          # Use pytesseract.image_to_string on the Image object
          page_text = pyt.image_to_string(img, lang='spa') # Spanish
          extracted_text += f"--- Page {i+1} ---\n{page_text}\n\n"
      return extracted_text

  except FileNotFoundError:
      print(f"Error: The PDF file was not found at {pdf_path_to_extract}")
  except Exception as e:
      print(f"An error occurred during PDF processing: {e}")


def insert_ocr_texts(df, pdf_dir, output_csv_path, save_frequency):
    """
    Insert ocr columns with none values. Iterates trough dataframe extracting text from
    PDFs using the function 'extract_text_pdf'
    - df is the dataframe with the pdfs links and where the text shold be inserted
    - pdf_dir is the directory where the pdfs are already downloaded
    - output_csv_path is the path where the dataframe will be saved
    - save_frequency is how often (in rows) the dataframe is saved
    """
    pdf_columns = [
        'texto_2debate',
        'texto_aprobado_pleno', 'objecion_parcial', 'objecion_total',
        'texto_definitivo_pleno', 'registro_oficial'
    ]

    # New text columns
    ocr_columns = [f"{col}_ocr" for col in pdf_columns]

    for ocr_col in ocr_columns:
        if ocr_col not in df.columns:
            df[ocr_col] = None

    # Iterate through each DataFrame row with a progress bar.
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Extracting OCR text"):
        for column in pdf_columns:
            flag_col = f"{column}_pdf_presente"
            ocr_col = f"{column}_ocr"

            # Verify if pdf is present
            if flag_col in row and row[flag_col] == 1 and (pd.isna(row[ocr_col]) or row[ocr_col] == ''):

                file_name = f"{row.get('id', 'no_id')}_{column}.pdf"
                file_path = os.path.join(pdf_dir, file_name)

                # Text extraction
                text = extract_text_pdf(file_path)

                # Insert text in columns
                if text is not None:
                   df.at[index, ocr_col] = text
                   tqdm.write(f"OCR successful for {file_name}")
                else:
                   tqdm.write(f"OCR failed or file not found for {file_name}")
                   df.at[index, ocr_col] = None
        if (index + 1) % save_frequency == 0:
          try:
              df.to_csv(output_csv_path, index=False)
              tqdm.write(f"Saved DataFrame to {output_csv_path} after processing row {index + 1}")
          except Exception as e:
              tqdm.write(f"Error saving DataFrame at row {index + 1}: {e}")

     # Save the final DataFrame after the loop finishes
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"Final DataFrame saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving final DataFrame: {e}")


    return df

### Run code with sample

In [ ]:
# Reset df_sample
csv_og_path = '/content/drive/MyDrive/Colab Notebooks/subset_final.csv'
df_subset_final = pd.read_csv(csv_og_path)

df_sample = df_subset_final.loc[df_subset_final['id'].isin({'AN-2014-54', 'AN-2014-55', 'AN-2014-56', 'AN-2014-74',
                                      'An-2014-98', 'AN-2014-100', 'AN-2014-101', 'AN-2014-103',
                                      'AN-2014-104', 'AN-2014-106'})].copy()
df_sample.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_sample.csv', index=False)

In [ ]:
csv_path_og = '/content/drive/MyDrive/Colab Notebooks/Bills_Ecuador09_24/subset_sample.csv'
df_sample = pd.read_csv(csv_path_og)
df_sample_con_ocr = copy.deepcopy(df_sample)
df_sample_con_ocr.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv', index=False)

In [ ]:
# Configure tessaract route
pyt.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# === Main Execution ===
csv_path = '/content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv'   # Change this path to your actual CSV file location
# Directory where the PDFs are stored
pdf_dir = '/content/drive/MyDrive/Colab Notebooks/Bills_Ecuador09_24/pdfs_sample'

output_csv_path = '/content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv'

# Saving frequency
save_frequency = 1

df_sample_con_ocr = pd.read_csv(csv_path)

print("Iniciando extracción de texto OCR...")
df_result = insert_ocr_texts(df_sample_con_ocr, pdf_dir, output_csv_path, save_frequency)
print("Extracción de texto OCR completada.")

try:
    df_result.to_csv(output_csv_path, index=False)
    print(f"DataFrame actualizado guardado en {output_csv_path}")
except Exception as e:
    print(f"Error al guardar el DataFrame actualizado: {e}")

print("\nPrimeras filas del DataFrame actualizado:")
display(df_result.head())

print("\nColumnas de texto OCR en el DataFrame del sample:")
ocr_cols = [col for col in df_result.columns if col.endswith('_ocr')]
print(df_result[ocr_cols].head())

df_result.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_sample_withOCR.csv', index=False)


Iniciando extracción de texto OCR...


Extracting OCR text:   0%|          | 0/10 [00:00<?, ?it/s]

Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 1
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 2
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 3
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 4
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 5
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 6
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 7
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 8
Saved DataFrame to /content/drive/MyDrive/Colab Notebooks/subset_sample_con_ocr.csv after processing row 9
Saved DataFrame to /content/drive/MyD

,fecha_presentacion,proyecto_ley,id,proponente,cargo,comision,estado,texto_proyecto_ley_pdf_presente,texto_proyecto_ley,texto_1debate_pdf_presente,...,matched_name,matching_score,coalition,party_2,texto_2debate_ocr,texto_aprobado_pleno_ocr,objecion_parcial_ocr,objecion_total_ocr,texto_definitivo_pleno_ocr,registro_oficial_ocr
0,2014-07-14,Proyecto de Ley Reformatoria del Código Orgáni...,AN-2014-104,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,Comisión de Justicia y Estructura del Estado,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,correa delgado pierina sara mercedes,86.0,UNES,RC,--- Page 1 ---\n \n\n \n\nYe a\n\nREPÚBLICA DE...,--- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...,--- Page 1 ---\n \n\n \n\n \n\n \n\n \n\n \...,NaN,"--- Page 1 ---\nQue,\n\nQue,\n\nQue,\n\nQue,\n...",--- Page 1 ---\n \n\n \n\nREGISTRO\n\nÓRGANO D...
1,2014-07-14,Proyecto de Ley Reformatoria a la Ley de Telec...,AN-2014-103,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,Comisión del Derecho al Trabajo y a la Segurid...,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,correa delgado pierina sara mercedes,86.0,UNES,RC,--- Page 1 ---\n \n\nA NACIONAE\n\n2 SAMBL ES ...,--- Page 1 ---\n \n\nASAMBLEA NACIONAL\n\ne\n\...,--- Page 1 ---\n \n\nPRESIDENCIA DE LA REPÚBLI...,NaN,"--- Page 1 ---\nQue,\n\nQue,\n\nQue,\n\nQue,\n...",--- Page 1 ---\nREGISTRO OFICIAL\n\nÓRGANO DEL...
2,2014-06-27,Proyecto de Ley Reformatoria a las Leyes Const...,AN-2014-101,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,Comisión de Soberanía Alimentaria y Desarrollo...,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,correa delgado pierina sara mercedes,86.0,UNES,RC,--- Page 1 ---\nREPÚBLICA DEL ECUADOR\n\nASAMB...,--- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...,--- Page 1 ---\n>\n>\n\nEs\n\nSy\n\nWe\n\nASAM...,NaN,"--- Page 1 ---\nQue,\n\nQue,\n\nQue,\n\nQue,\n...",--- Page 1 ---\n \n\nREGISTRO OFICIAL\n\nÓRGAN...
3,2014-06-27,Proyecto de Ley Reformatoria a la Ley Orgánica...,AN-2014-100,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,"Comisión de Educación, Cultura, Ciencia, Tecno...",Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,correa delgado pierina sara mercedes,86.0,UNES,RC,--- Page 1 ---\n \n\n \n\n \n\n \n\n \n\n \n\...,--- Page 1 ---\n \n\n \n\nREPÚBLICA DEL ECUAD...,--- Page 1 ---\n \n\n_CARONDELET__\n\nOficio...,NaN,"--- Page 1 ---\nQue,\n\nQue,\n\nQue,\n\nQue,\n...",--- Page 1 ---\n \n \n\n \n\nAño lll - N2 572...
4,2014-06-25,Proyecto de Código Orgánico Monetario y FiNA...,An-2014-98,correa delgado rafael,PRESIDENTE DE LA REPÚBLICA DEL ECUADOR,Comisión de Régimen Económico y Tributario y s...,Publicado /,1,https://ppless.asambleanacional.gob.ec/alfresc...,1,...,correa delgado pierina sara mercedes,86.0,UNES,RC,--- Page 1 ---\nMe\nAÑAMBLIA NaCHINar\nPoroara...,--- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\n\n...,--- Page 1 ---\nMe\n\nASARÍBLA Nacional\n\nFLs...,NaN,--- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\n\n...,--- Page 1 ---\nf REGISTRO OFICIAL\n\nÓRGANO D...



Columnas de texto OCR en el DataFrame del sample:
                                   texto_2debate_ocr  \
0  --- Page 1 ---\n \n\n \n\nYe a\n\nREPÚBLICA DE...   
1  --- Page 1 ---\n \n\nA NACIONAE\n\n2 SAMBL ES ...   
2  --- Page 1 ---\nREPÚBLICA DEL ECUADOR\n\nASAMB...   
3  --- Page 1 ---\n \n\n  \n\n \n\n \n\n \n\n \n\...   
4  --- Page 1 ---\nMe\nAÑAMBLIA NaCHINar\nPoroara...   

                            texto_aprobado_pleno_ocr  \
0  --- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...   
1  --- Page 1 ---\n \n\nASAMBLEA NACIONAL\n\ne\n\...   
2  --- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...   
3  --- Page 1 ---\n \n\n  \n\nREPÚBLICA DEL ECUAD...   
4  --- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\n\n...   

                                objecion_parcial_ocr  objecion_total_ocr  \
0  --- Page 1 ---\n \n\n \n\n   \n\n \n\n  \n\n \...                 NaN   
1  --- Page 1 ---\n \n\nPRESIDENCIA DE LA REPÚBLI...                 NaN   
2  --- Page 1 ---\n>\n>\n\nEs\n\nSy\n\nWe\n\nAS

In [ ]:
print(df_sample_con_ocr.iloc[:, 27:])

  party_2                                  texto_2debate_ocr  \
0      RC  --- Page 1 ---\n \n\n \n\nYe a\n\nREPÚBLICA DE...   
1      RC  --- Page 1 ---\n \n\nA NACIONAE\n\n2 SAMBL ES ...   
2      RC  --- Page 1 ---\nREPÚBLICA DEL ECUADOR\n\nASAMB...   
3      RC  --- Page 1 ---\n \n\n  \n\n \n\n \n\n \n\n \n\...   
4      RC  --- Page 1 ---\nMe\nAÑAMBLIA NaCHINar\nPoroara...   
5    CREO  --- Page 1 ---\n \n\n. Y ,\nMe ASAMBLEA NACION...   
6      RC  --- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...   
7    CREO  --- Page 1 ---\nCOMISION DE JUSTICIA ]\nY ESTR...   
8     PSP  --- Page 1 ---\nCOMISION DE JUSTICIA ]\nY ESTR...   
9      RC  --- Page 1 ---\n7H 203397\n\na TACO\nASAMEL mA...   

                            texto_aprobado_pleno_ocr  \
0  --- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...   
1  --- Page 1 ---\n \n\nASAMBLEA NACIONAL\n\ne\n\...   
2  --- Page 1 ---\n \n\nREPÚBLICA DEL ECUADOR\nAS...   
3  --- Page 1 ---\n \n\n  \n\nREPÚBLICA DEL ECUAD...   
4  --- Page 1 -

In [ ]:
## Merge asambleistas_review with subset_con_partido
csv_path_asambleistas_review = '/content/drive/MyDrive/Colab Notebooks/Bills_Ecuador09_24/asambleistas_09_24_review.csv'
csv_path_subset = '/content/drive/MyDrive/Colab Notebooks/subset_con_partido.csv'
df_asambleistas_review = pd.read_csv(csv_path_asambleistas_review)
df_subset = pd.read_csv(csv_path_subset)

df_review = df_asambleistas_review[['id', 'coalition', 'party_2']]
df_merged = pd.merge(df_subset, df_review,
                     on='id',
                     how='left',
                     suffixes=('_original', '_review'))

df_subset['coalition'] = df_merged['coalition_review'].combine_first(df_merged['coalition_original'])
df_subset['party_2'] = df_merged['party_2_review'].combine_first(df_merged['party_2_original'])

df_subset.to_csv('/content/drive/MyDrive/Colab Notebooks/subset_.csv', index=False)

In [ ]:
df_subset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/subset_sample.csv')
df_subset.head()
